In [38]:
#!python3.11 -m pip install tools
#!pip -V # shows where pip installs 
#import sysconfig; print(sysconfig.get_paths()["purelib"]) # shows where python looks for packages 

from src.scrape import * 
import pandas as pd
from pathlib import Path
import re
import nltk
import nltk.data
import pandas as pd 
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/KattPaint/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Web Scrape Text

Code from: https://github.com/oscarsuen/beige-book/tree/master/txt/1970/05

In [2]:
# scrape data into folder - ONLY RUN ONCE 
scrape(skip=False)

2010 01 at 

/Users/KattPaint/Desktop/DATASCI266/10YT-Prediction-LSTM-NLP-Beige-Book/src/scrape.py:30: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 30 of the file /Users/KattPaint/Desktop/DATASCI266/10YT-Prediction-LSTM-NLP-Beige-Book/src/scrape.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(r.text)


y
2010 01 bo y
2010 01 ch y
2010 01 cl y
2010 01 da y
2010 01 kc y
2010 01 mi y
2010 01 ny y
2010 01 ph y
2010 01 ri y
2010 01 sf y
2010 01 sl y
2010 01 su y
2010 02 at n
2010 02 bo n
2010 02 ch n
2010 02 cl n
2010 02 da n
2010 02 kc n
2010 02 mi n
2010 02 ny n
2010 02 ph n
2010 02 ri n
2010 02 sf n
2010 02 sl n
2010 02 su n
2010 03 at y
2010 03 bo y
2010 03 ch y
2010 03 cl y
2010 03 da y
2010 03 kc y
2010 03 mi y
2010 03 ny y
2010 03 ph y
2010 03 ri y
2010 03 sf y
2010 03 sl y
2010 03 su y
2010 04 at y
2010 04 bo y
2010 04 ch y
2010 04 cl y
2010 04 da y
2010 04 kc y
2010 04 mi y
2010 04 ny y
2010 04 ph y
2010 04 ri y
2010 04 sf y
2010 04 sl y
2010 04 su y
2010 05 at n
2010 05 bo n
2010 05 ch n
2010 05 cl n
2010 05 da n
2010 05 kc n
2010 05 mi n
2010 05 ny n
2010 05 ph n
2010 05 ri n
2010 05 sf n
2010 05 sl n
2010 05 su n
2010 06 at y
2010 06 bo y
2010 06 ch y
2010 06 cl y
2010 06 da y
2010 06 kc y
2010 06 mi y
2010 06 ny y
2010 06 ph y
2010 06 ri y
2010 06 sf y
2010 06 sl y
2010 06 su

# Clean Text 

Code from: https://github.com/oscarsuen/beige-book/tree/master/txt/1970/05

In [39]:
# get files 
files = []

regions = ("at", "bo", "ch", "cl", "da", "kc", "mi", "ny", "ph", "ri", "sf", "sl", "su")
for year in range(1970, 2025):
    for month in range(1, 13): 
        for region in regions:
            file = f"../data/beige_books/txt/{year}/{month:02d}/{year}-{month:02d}-{region}.txt"
            if Path(file).is_file():
                files.append(file)

len(files)

6131

In [40]:
# clean each file and re-write 
for filename in files:
    with open(filename, "r") as f:
            old_txt = f.read()
    new_txt = old_txt

    new_txt = re.sub(r"For more information about District economic conditions,? visit: URL", "", new_txt)
    new_txt = new_txt.replace("%-", " percent to ").replace("%", " percent")
    new_txt = new_txt.replace(" & ", " and ")
    new_txt = new_txt.replace("=", " equals ") # only one instance

    new_txt = re.sub(r"[<>~*]", "", new_txt)
    new_txt = re.sub(r"\-\-+", " , ", new_txt)
    new_txt = re.sub(r"\?(?=[\w])", "? ", new_txt).replace(" ?", "?")
    new_txt = re.sub(r"\s+,", ",", new_txt).replace(",,", ",")
    new_txt = re.sub(r"\s+\.(?=[^0-9])", " ", new_txt)
    new_txt = new_txt.replace("...", " ")
    new_txt = new_txt.replace("..", ".").replace(",.", ",") # ".," is legitimate
    new_txt = new_txt.replace("[", "").replace("]", "") # parentheses are legitimate

    new_txt = re.sub(r"\s+", " ", new_txt)
    new_txt = new_txt.strip()
    with open(filename, "w") as f:
        f.write(new_txt)

# Create Dataframe for Topic Modeling

In [112]:
# initialize data frame to store data 
df = pd.DataFrame(columns = ['year', 'month', 'region', 'sentence'])

# sentence splitter
m = nltk.data.load('tokenizers/punkt/english.pickle')

for filename in files:
    i = files.index(filename)
    if i % 500 == 0:
        print(filename, i)
        
    year, month, region = filename.split('/')[6].split('.')[0].split('-')
    
    with open(filename, "r") as f:
        txt = f.read()
        
    txt = m.tokenize(txt)
    txt = pd.DataFrame(txt, columns=['sentence'])
    txt.insert(0, 'region', region)
    txt.insert(0, 'month', month)
    txt.insert(0, 'year', year)
    
    df = pd.concat([df, txt], axis=0)

../data/beige_books/txt/1970/05/1970-05-at.txt 0
../data/beige_books/txt/1973/08/1973-08-ny.txt 500
../data/beige_books/txt/1976/11/1976-11-bo.txt 1000
../data/beige_books/txt/1980/05/1980-05-ri.txt 1500
../data/beige_books/txt/1985/01/1985-01-mi.txt 2000
../data/beige_books/txt/1989/12/1989-12-bo.txt 2500
../data/beige_books/txt/1994/09/1994-09-ny.txt 3000
../data/beige_books/txt/1999/08/1999-08-bo.txt 3500
../data/beige_books/txt/2004/04/2004-04-ny.txt 4000
../data/beige_books/txt/2009/03/2009-03-bo.txt 4500
../data/beige_books/txt/2013/12/2013-12-ny.txt 5000
../data/beige_books/txt/2018/10/2018-10-mi.txt 5500
../data/beige_books/txt/2023/07/2023-07-su.txt 6000


In [113]:
df.head()

,year,month,region,sentence
0,1970,05,at,The mood of our directors varies from pessimis...
1,1970,05,at,"If any consensus exists, it is that business a..."
2,1970,05,at,Many major economic indices should drift downw...
3,1970,05,at,"In the pessimistic vein, a leading department ..."
4,1970,05,at,The store reported that labor costs were up 8 ...


In [114]:
df.tail()

,year,month,region,sentence
83,2024,10,su,San Francisco Economic activity was steady.
84,2024,10,su,"Labor availability improved further, and wages..."
85,2024,10,su,Overall prices were largely stable.
86,2024,10,su,Retail sales and activity in manufacturing and...
87,2024,10,su,"Demand for business services improved, while c..."


In [116]:
df.shape

(327504, 4)

In [ ]:
# add date time column 
df.insert(2, 'day', 1)
dates = df.year.astype(str) + "-" + df.month.astype(str) + "-" + df.day.astype(str)
dates = pd.to_datetime(dates,format='%Y-%m-%d')
df.insert(0, 'time_index', dates)
df = df.drop(['year', 'month', 'day'], axis=1)

# Export Data

In [131]:
df.to_csv('../data/beige_books_1970_2024.csv', encoding='utf-8', index=True)